# C interpreter

### 1. First aproximation

Design a agrammar using an upward translation that accepts the following inputs from the C language:

- Assignment statements `(a = 5, a = b = c = 5)`
- Comparison operators `(==, <=, >=, !=)`
- Logical Operators `(&&, ||, !)`
- Arithmetic operators `(+, - , *, / and unary minus)`
- Variables and numerical constants `(a, 5)`



## C operator precedence

The C operator predecence that we have to implement in our grammar is in this precedence order:

1. Variables and numerical constants
2. Unary minus and Logical NOT
3. Arithmetic operators    
    - Multiplication, division, and reminder
    - addition and subtraction
4. Logical operators
    - equals and not equals
    - Logical AND 
    - Logical OR
5.  Assignment statements